In [1]:
import os
import json
import uuid
import time
import requests
from tqdm import tqdm
import pandas as pd

In [2]:
def run_notebook_test(run_label, api_url, course_id, input_file, moodle_token=None, output_folder=None, mode=None):
    #output_folder = "philosophy"
    os.makedirs(output_folder, exist_ok=True)
    file_tag = f"_{run_label}" if run_label else ""
    filename = f"results{file_tag}.json"
    output_path = os.path.join(output_folder, filename)
    
    print(f"Test Start: '{run_label}'")
    print(f"Results will be saved in: {output_path}")
    
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            test_data = json.load(f)
    except FileNotFoundError:
        print(f"ERROR: Input file not found: {input_file}")
        return None

    results = []
    headers = {"Content-Type": "application/json"}
    if moodle_token:
        headers["Authorization"] = f"Bearer {moodle_token}"
    
    n = 0
    for item in tqdm(test_data, desc="Test progress"):
        n += 1
        question = item.get("question")
        ground_truth = item.get("ground_truth")
        
        if not question: 
            continue

        session_id = str(uuid.uuid4())
        
        payload = {
            "message": question,
            "courseId": str(course_id), 
            "sessionId": session_id,
            "history": [],
            "mode": mode
        }

        start_time = time.time()
        bot_answer = ""
        contexts = []
        error_msg = None
        duration = 0 

        try:
            response = requests.post(api_url, json=payload, headers=headers, timeout=180)
            
            if response.status_code == 200:
                duration = time.time() - start_time
                resp_data = response.json()
                bot_answer = resp_data.get("response", "")
                raw_sources = resp_data.get("sources", [])
                if raw_sources:
                    for src in raw_sources:
                        text_content = src.get("chunkText", "")
                        if not text_content:
                            text_content = str(src)
                        contexts.append(text_content)
            else:
                error_msg = f"HTTP {response.status_code}"
                bot_answer = f"ERROR: {error_msg}"
                if response.status_code != 422: 
                    print(f"\nSERVER ERROR: {response.status_code} for question: {question}")

        except Exception as e:
            error_msg = str(e)
            bot_answer = "ERROR: Timeout/Connection"
            duration = 180
            print(f"\n Timeout or Connection Error: {e}")
        
        if n == 1:
            print(f"\n First context preview (truncated): {str(contexts)[:200]}...")

        results.append({
            "config_label": run_label,
            "question": question,
            "answer": bot_answer,
            "contexts": contexts,
            "ground_truth": ground_truth,
            "latency": round(duration, 2),
            "error": error_msg
        })

        time.sleep(2.0)

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"Success! JSON saved at: {output_path}")
    except Exception as e:
        print(f"Error saving JSON file: {e}")

    return results

In [3]:
def run_pipeline_test(run_label, api_url, course_id, input_file, moodle_token=None, output_folder="philosophy"):
    results_list = run_notebook_test(
        run_label=run_label,
        api_url=api_url,
        course_id=course_id,
        input_file=input_file,
        moodle_token=moodle_token,
        output_folder=output_folder
    )
    if results_list is not None:
        df_results = pd.DataFrame(results_list)
        print(f"\nŚredni czas odpowiedzi: {df_results['latency'].mean():.2f}s")
        display(df_results[['question', 'answer', 'latency', 'contexts']].head())
    return df_results    

# statistics course

In [4]:
API_URL = "https://chatbot-backend-production-4b66.up.railway.app/chat"  # Lub adres Railway
TEST_COURSE_ID = "11"                      # ID kursu z materiałami
MOODLE_TOKEN = '9f66de570d71c9b474cce459dd02a2bd'               # Opcjonalnie token
INPUT_FILE = "question_answer_statistics.json"

In [23]:
RUN_LABEL = "statistics_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist08_temp01" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist08_temp01'
Results will be saved in: statistics\results_statistics_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist08_temp01.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A ...


Test progress: 100%|██████████| 15/15 [11:23<00:00, 45.56s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist08_temp01.json

Średni czas odpowiedzi: 43.55s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"According to the course materials, Yule revive...",22.25,[cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic ...
1,What are the three components that make up the...,"Okay, let's break down the components of the e...",25.38,[and out-of-sample) for supervised learning\n\...
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",49.64,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"According to the course materials, the `e1071`...",150.84,[http://ocw.mit.edu\n\n15.097 Prediction: Mach...
4,What specific node impurity measure is used by...,"Based on the provided context, the CART (Class...",35.05,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_1000_overlap_200_voya...,Who revived the term 'logistic' in 1925 to des...,"According to the course materials, Yule revive...",[cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic ...,The name 'logistic' was revived by Yule in a p...,22.25,None
1,statistics_results_chunk_1000_overlap_200_voya...,What are the three components that make up the...,"Okay, let's break down the components of the e...",[and out-of-sample) for supervised learning\n\...,According to the Bias-Variance decomposition t...,25.38,None
2,statistics_results_chunk_1000_overlap_200_voya...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,49.64,None
3,statistics_results_chunk_1000_overlap_200_voya...,Which R package is required to implement Suppo...,"According to the course materials, the `e1071`...",[http://ocw.mit.edu\n\n15.097 Prediction: Mach...,The 'e1071' package is the additional R packag...,150.84,None
4,statistics_results_chunk_1000_overlap_200_voya...,What specific node impurity measure is used by...,"Based on the provided context, the CART (Class...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...","For classification splitting, the CART algorit...",35.05,None
5,statistics_results_chunk_1000_overlap_200_voya...,Explain the two iterative steps of the K-Means...,"Okay, I can help you understand the K-Means al...",[tall the arules package and load it. See Sect...,"Before starting, the user must choose the valu...",47.48,None
6,statistics_results_chunk_1000_overlap_200_voya...,How does the Naïve Bayes algorithm handle high...,"Okay, I can help you understand how the Naïve ...",[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,29.94,None
7,statistics_results_chunk_1000_overlap_200_voya...,Describe the logic behind using the 'Gain Rati...,The course material explains that while Inform...,[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,Information Gain tends to favor attributes tha...,31.03,None
8,statistics_results_chunk_1000_overlap_200_voya...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down why Lemma 1 tells us th...",[Clustering\n\nMIT 15.097 Course Notes Cynthia...,Lemma 1 proves that for any set C and any repr...,50.57,None
9,statistics_results_chunk_1000_overlap_200_voya...,What are the three options C4.5 considers when...,"Okay, I can help you with that! According to t...",[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,"At each node, C4.5 recursively evaluates three...",16.10,None


In [24]:
RUN_LABEL = "statistics_results_chunk_1000_overlap_200_voyagelarge2_gemini20_flash_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_1000_overlap_200_voyagelarge2_gemini20_flash_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_1000_overlap_200_voyagelarge2_gemini20_flash_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['http://ocw.mit.edu\n\n15.097 Prediction: Machine Learning and Statistics\n\nSpring 2012\n\nFor information about citing these materials or our Terms of Use, visit: http://ocw.mit.edu/terms.', 'Regul...


Test progress: 100%|██████████| 15/15 [11:28<00:00, 45.88s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_1000_overlap_200_voyagelarge2_gemini20_flash_topk10_dist08_temp03.json

Średni czas odpowiedzi: 43.87s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, Yule revived th...",17.09,[http://ocw.mit.edu\n\n15.097 Prediction: Mach...
1,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...",19.15,[http://ocw.mit.edu\n\n15.097 Prediction: Mach...
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",51.54,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,Based on the provided text:\n\n* **Support V...,151.30,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"Based on the provided context, the CART (Class...",25.25,"[o play tennis, analyze C-section risk, etc.\n..."


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_1000_overlap_200_voya...,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, Yule revived th...",[http://ocw.mit.edu\n\n15.097 Prediction: Mach...,The name 'logistic' was revived by Yule in a p...,17.09,None
1,statistics_results_chunk_1000_overlap_200_voya...,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...",[http://ocw.mit.edu\n\n15.097 Prediction: Mach...,According to the Bias-Variance decomposition t...,19.15,None
2,statistics_results_chunk_1000_overlap_200_voya...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,51.54,None
3,statistics_results_chunk_1000_overlap_200_voya...,Which R package is required to implement Suppo...,Based on the provided text:\n\n* **Support V...,[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,151.30,None
4,statistics_results_chunk_1000_overlap_200_voya...,What specific node impurity measure is used by...,"Based on the provided context, the CART (Class...","[o play tennis, analyze C-section risk, etc.\n...","For classification splitting, the CART algorit...",25.25,None
5,statistics_results_chunk_1000_overlap_200_voya...,Explain the two iterative steps of the K-Means...,"Okay, I can help you understand the K-Means al...",[ic regression. Using the same notation as in ...,"Before starting, the user must choose the valu...",54.52,None
6,statistics_results_chunk_1000_overlap_200_voya...,How does the Naïve Bayes algorithm handle high...,That's a great question! You're right to focus...,[http://ocw.mit.edu\n\n15.097 Prediction: Mach...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,33.78,None
7,statistics_results_chunk_1000_overlap_200_voya...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the benefit of using 'Gain...","[o play tennis, analyze C-section risk, etc.\n...",Information Gain tends to favor attributes tha...,30.15,None
8,statistics_results_chunk_1000_overlap_200_voya...,"According to Lemma 1 in the clustering notes, ...",Lemma 1 provides a way to understand why placi...,[¯y = Ey[y|x].\n\nSo we know for each x what t...,Lemma 1 proves that for any set C and any repr...,48.90,None
9,statistics_results_chunk_1000_overlap_200_voya...,What are the three options C4.5 considers when...,C4.5 considers three options when deciding whe...,"[o play tennis, analyze C-section risk, etc.\n...","At each node, C4.5 recursively evaluates three...",15.84,None


In [25]:
RUN_LABEL = "statistics_results_chunk_1000_overlap_200_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_1000_overlap_200_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_1000_overlap_200_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A ...


Test progress: 100%|██████████| 15/15 [09:21<00:00, 37.46s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_1000_overlap_200_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03.json

Średni czas odpowiedzi: 35.45s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",18.17,[cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic ...
1,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...",24.68,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",47.59,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,Based on the provided course materials:\n\n* ...,28.66,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",35.56,[Example of how K-Means could converge to the ...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_1000_overlap_200_text...,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",[cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic ...,The name 'logistic' was revived by Yule in a p...,18.17,None
1,statistics_results_chunk_1000_overlap_200_text...,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,24.68,None
2,statistics_results_chunk_1000_overlap_200_text...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,47.59,None
3,statistics_results_chunk_1000_overlap_200_text...,Which R package is required to implement Suppo...,Based on the provided course materials:\n\n* ...,[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,28.66,None
4,statistics_results_chunk_1000_overlap_200_text...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[Example of how K-Means could converge to the ...,"For classification splitting, the CART algorit...",35.56,None
5,statistics_results_chunk_1000_overlap_200_text...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...",[tall the arules package and load it. See Sect...,"Before starting, the user must choose the valu...",45.00,None
6,statistics_results_chunk_1000_overlap_200_text...,How does the Naïve Bayes algorithm handle high...,That's a great question! You're right to think...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,30.28,None
7,statistics_results_chunk_1000_overlap_200_text...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the logic behind using the...",[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,Information Gain tends to favor attributes tha...,39.35,None
8,statistics_results_chunk_1000_overlap_200_text...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down how Lemma 1 justifies t...",[Clustering\n\nMIT 15.097 Course Notes Cynthia...,Lemma 1 proves that for any set C and any repr...,38.54,None
9,statistics_results_chunk_1000_overlap_200_text...,What are the three options C4.5 considers when...,C4.5 considers these three options when decidi...,[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,"At each node, C4.5 recursively evaluates three...",37.43,None


In [26]:
RUN_LABEL = "statistics_results_chunk_1000_overlap_200_text_to_embedding_large_gemini20_flash_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_1000_overlap_200_text_to_embedding_large_gemini20_flash_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_1000_overlap_200_text_to_embedding_large_gemini20_flash_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A ...


Test progress: 100%|██████████| 15/15 [09:18<00:00, 37.20s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_1000_overlap_200_text_to_embedding_large_gemini20_flash_topk10_dist08_temp03.json

Średni czas odpowiedzi: 35.19s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",20.61,[cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic ...
1,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...",30.65,[Rtest is also called the true risk or the tes...
2,"In the context of the Apriori algorithm, what ...","The downward closure property of support, in t...",39.23,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,Based on the provided course materials:\n\n* ...,30.26,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",49.30,[I have to warn you that C4.5 and CART are not...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_1000_overlap_200_text...,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",[cw.mit.edu/fairuse.\n\n−yif(xi)\n• “logistic ...,The name 'logistic' was revived by Yule in a p...,20.61,None
1,statistics_results_chunk_1000_overlap_200_text...,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...",[Rtest is also called the true risk or the tes...,According to the Bias-Variance decomposition t...,30.65,None
2,statistics_results_chunk_1000_overlap_200_text...,"In the context of the Apriori algorithm, what ...","The downward closure property of support, in t...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,39.23,None
3,statistics_results_chunk_1000_overlap_200_text...,Which R package is required to implement Suppo...,Based on the provided course materials:\n\n* ...,[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,30.26,None
4,statistics_results_chunk_1000_overlap_200_text...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[I have to warn you that C4.5 and CART are not...,"For classification splitting, the CART algorit...",49.30,None
5,statistics_results_chunk_1000_overlap_200_text...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...",[tall the arules package and load it. See Sect...,"Before starting, the user must choose the valu...",24.59,None
6,statistics_results_chunk_1000_overlap_200_text...,How does the Naïve Bayes algorithm handle high...,The course materials explain that Naïve Bayes ...,[K-Nearest Neighbors\n\n• Classify using the m...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,43.95,None
7,statistics_results_chunk_1000_overlap_200_text...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the reasoning behind using...",[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,Information Gain tends to favor attributes tha...,27.61,None
8,statistics_results_chunk_1000_overlap_200_text...,"According to Lemma 1 in the clustering notes, ...",Lemma 1 provides a way to understand why placi...,[Clustering\n\nMIT 15.097 Course Notes Cynthia...,Lemma 1 proves that for any set C and any repr...,43.45,None
9,statistics_results_chunk_1000_overlap_200_text...,What are the three options C4.5 considers when...,C4.5 considers three options when deciding whe...,"[o play tennis, analyze C-section risk, etc.\n...","At each node, C4.5 recursively evaluates three...",35.16,None


In [27]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:48<00:00, 39.26s/it]


Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03.json

Średni czas odpowiedzi: 37.24s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",32.37,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,"Okay, I can help you understand the decomposit...",24.86,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",48.59,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",32.70,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",39.00,[istance) to the centroid (want it to be small...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,32.37,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you understand the decomposit...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,24.86,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,48.59,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,32.70,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[istance) to the centroid (want it to be small...,"For classification splitting, the CART algorit...",39.00,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...","[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",37.71,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,"That's a great question! You're right, high-di...",[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,42.76,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The course materials explain that while Inform...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,23.46,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down how Lemma 1 shows that ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,46.54,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with that! According to t...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",27.14,None


In [28]:
RUN_LABEL = "statistics_results_chunk_2000_overlap_400_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_2000_overlap_400_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_2000_overlap_400_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['> norm_vec <- rnorm(n=10, mean=5, sd=2)\n> exp_vec <- rexp(n=100, rate=3)\n> pois_vec <- rpois(n=50, lambda=6)\n> unif_vec <- runif(n=20, min=1, max=9)\n> bin_vec <- rbinom(n=20, size=1000, prob=0.7...


Test progress: 100%|██████████| 15/15 [09:44<00:00, 38.99s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_2000_overlap_400_text_to_embedding_small_gemini20_flash_topk10_dist08_temp03.json

Średni czas odpowiedzi: 36.97s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, the name ""logistic...",18.84,"[> norm_vec <- rnorm(n=10, mean=5, sd=2)\n> ex..."
1,What are the three components that make up the...,"Okay, great question!\n\nAccording to the cour...",30.90,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",51.76,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,Based on the provided course materials:\n\n* ...,21.22,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"Based on the course materials, the CART (Class...",46.03,[Rule Mining and the Apriori Algorithm\n\nMIT ...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_2000_overlap_400_text...,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, the name ""logistic...","[> norm_vec <- rnorm(n=10, mean=5, sd=2)\n> ex...",The name 'logistic' was revived by Yule in a p...,18.84,None
1,statistics_results_chunk_2000_overlap_400_text...,What are the three components that make up the...,"Okay, great question!\n\nAccording to the cour...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,30.90,None
2,statistics_results_chunk_2000_overlap_400_text...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,51.76,None
3,statistics_results_chunk_2000_overlap_400_text...,Which R package is required to implement Suppo...,Based on the provided course materials:\n\n* ...,[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,21.22,None
4,statistics_results_chunk_2000_overlap_400_text...,What specific node impurity measure is used by...,"Based on the course materials, the CART (Class...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,"For classification splitting, the CART algorit...",46.03,None
5,statistics_results_chunk_2000_overlap_400_text...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...",[ost of the following algorithms (as well as l...,"Before starting, the user must choose the valu...",24.86,None
6,statistics_results_chunk_2000_overlap_400_text...,How does the Naïve Bayes algorithm handle high...,That's a great question! The Naïve Bayes algor...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,49.50,None
7,statistics_results_chunk_2000_overlap_400_text...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the rationale behind using...",[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,Information Gain tends to favor attributes tha...,28.21,None
8,statistics_results_chunk_2000_overlap_400_text...,"According to Lemma 1 in the clustering notes, ...","Lemma 1 states: cost(C; z) = cost(C, mean(C)) ...",[It turns out that the f ∗ that minimizes the ...,Lemma 1 proves that for any set C and any repr...,54.84,None
9,statistics_results_chunk_2000_overlap_400_text...,What are the three options C4.5 considers when...,C4.5 considers these three options when decidi...,[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,"At each node, C4.5 recursively evaluates three...",22.87,None


In [29]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk15_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk15_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk15_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:45<00:00, 39.02s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk15_dist08_temp03.json

Średni czas odpowiedzi: 37.01s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, the name ""logis...",19.59,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...",32.35,"[ β1X1 + β2X2 + · · · + βkXk + ε,\n\nwhere ε i..."
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",44.22,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,Based on the provided text:\n\n* **Support V...,33.96,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",46.11,[Rule Mining and the Apriori Algorithm\n\nMIT ...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, the name ""logis...",[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,19.59,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you with that!\n\nBased on th...","[ β1X1 + β2X2 + · · · + βkXk + ε,\n\nwhere ε i...",According to the Bias-Variance decomposition t...,32.35,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,44.22,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,Based on the provided text:\n\n* **Support V...,[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,33.96,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,"For classification splitting, the CART algorit...",46.11,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...","[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",27.30,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,The course material explains that Naïve Bayes ...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,41.34,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the logic behind using the...",[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,Information Gain tends to favor attributes tha...,35.93,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...",Lemma 1 provides a way to understand why placi...,[Clustering\n\nMIT 15.097 Course Notes Cynthia...,Lemma 1 proves that for any set C and any repr...,38.52,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with that! According to t...",[Decision Trees\n\nMIT 15.097 Course Notes Cyn...,"At each node, C4.5 recursively evaluates three...",25.25,None


In [ ]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk5_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk5_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk5_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['Logistic Regression\n\nMIT 15.097 Course Notes Cynthia Rudin\n\nThanks to Ashia Wilson Credit: J.S. Cramer’s “The Origin of Logistic Regression”\n\nOrigins: 19th Century.\n\n• Studying growth of pop...


Test progress: 100%|██████████| 15/15 [09:16<00:00, 37.13s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk5_dist08_temp03.json

Średni czas odpowiedzi: 35.12s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, the name ""logis...",23.37,[Logistic Regression\n\nMIT 15.097 Course Note...
1,What are the three components that make up the...,"Okay, I can help you understand the decomposit...",23.41,[It turns out that the f ∗ that minimizes the ...
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",46.94,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",21.53,"[nclude “base,” “utils,” “graphics,” and “stat..."
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",50.00,[I have to warn you that C4.5 and CART are not...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, the name ""logis...",[Logistic Regression\n\nMIT 15.097 Course Note...,The name 'logistic' was revived by Yule in a p...,23.37,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you understand the decomposit...",[It turns out that the f ∗ that minimizes the ...,According to the Bias-Variance decomposition t...,23.41,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,46.94,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...","[nclude “base,” “utils,” “graphics,” and “stat...",The 'e1071' package is the additional R packag...,21.53,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[I have to warn you that C4.5 and CART are not...,"For classification splitting, the CART algorit...",50.00,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can help you understand the K-Means al...",[an objects assigned to different clusters. We...,"Before starting, the user must choose the valu...",29.53,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,"Okay, I can help you understand how the Naïve ...",[Na¨ıve Bayes\n\nMIT 15.097 Course Notes Cynth...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,36.32,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the rationale behind using...",[Two examples of de cision trees. Opti on\n\nf...,Information Gain tends to favor attributes tha...,34.93,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down why Lemma 1 tells us th...","[While we’re analyzing, we’ll need to consider...",Lemma 1 proves that for any set C and any repr...,36.82,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with that! Based on the c...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",30.85,None


In [31]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist06_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist06_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist06_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['Logistic Regression\n\nMIT 15.097 Course Notes Cynthia Rudin\n\nThanks to Ashia Wilson Credit: J.S. Cramer’s “The Origin of Logistic Regression”\n\nOrigins: 19th Century.\n\n• Studying growth of pop...


Test progress: 100%|██████████| 15/15 [09:48<00:00, 39.21s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist06_temp03.json

Średni czas odpowiedzi: 37.19s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, the name ""logistic...",19.03,[Logistic Regression\n\nMIT 15.097 Course Note...
1,What are the three components that make up the...,"Okay, I can help you with that! Based on the c...",31.28,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",51.82,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",26.53,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",49.83,[I have to warn you that C4.5 and CART are not...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, the name ""logistic...",[Logistic Regression\n\nMIT 15.097 Course Note...,The name 'logistic' was revived by Yule in a p...,19.03,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you with that! Based on the c...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,31.28,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,51.82,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,26.53,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[I have to warn you that C4.5 and CART are not...,"For classification splitting, the CART algorit...",49.83,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can help you understand the K-Means al...","[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",32.22,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,"Okay, I can explain how the Naïve Bayes algori...",[Na¨ıve Bayes\n\nMIT 15.097 Course Notes Cynth...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,42.49,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The course materials explain that while Inform...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,24.77,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down how Lemma 1 shows that ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,52.12,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can explain the three options C4.5 con...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",21.51,None


In [32]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist05_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist05_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist05_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['Logistic Regression\n\nMIT 15.097 Course Notes Cynthia Rudin\n\nThanks to Ashia Wilson Credit: J.S. Cramer’s “The Origin of Logistic Regression”\n\nOrigins: 19th Century.\n\n• Studying growth of pop...


Test progress: 100%|██████████| 15/15 [09:46<00:00, 39.07s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist05_temp03.json

Średni czas odpowiedzi: 37.06s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, the name ""logis...",32.70,[Logistic Regression\n\nMIT 15.097 Course Note...
1,What are the three components that make up the...,"Okay, let's break down the decomposition of th...",24.16,[It turns out that the f ∗ that minimizes the ...
2,"In the context of the Apriori algorithm, what ...",The downward closure property of support is a ...,48.32,[We want to find all strong rules. These are r...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",34.54,"[nclude “base,” “utils,” “graphics,” and “stat..."
4,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,30.95,[CART - Classification and Regression Trees (B...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, the name ""logis...",[Logistic Regression\n\nMIT 15.097 Course Note...,The name 'logistic' was revived by Yule in a p...,32.70,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, let's break down the decomposition of th...",[It turns out that the f ∗ that minimizes the ...,According to the Bias-Variance decomposition t...,24.16,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...",The downward closure property of support is a ...,[We want to find all strong rules. These are r...,Downward closure is a monotonicity property st...,48.32,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...","[nclude “base,” “utils,” “graphics,” and “stat...",The 'e1071' package is the additional R packag...,34.54,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,[CART - Classification and Regression Trees (B...,"For classification splitting, the CART algorit...",30.95,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Alright, let's break down the K-Means algorith...",[The K-Means Algorithm\n\nChoose the value of ...,"Before starting, the user must choose the valu...",37.63,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,"Okay, I can help you understand how the Naive ...",[s\nover an unknown distribution over X × Y.\n...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,44.35,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the rationale behind using...","[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,31.14,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down why Lemma 1 tells us th...","[While we’re analyzing, we’ll need to consider...",Lemma 1 proves that for any set C and any repr...,39.09,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can clarify the options C4.5 considers...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",42.10,None


In [33]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist045_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist045_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist045_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['He published in 3 papers between 1838 and 1847. The first paper demonstrated that the curve agrees very well with the actual course of the population in France, Belgium, Essex, and Russia for period...


Test progress: 100%|██████████| 15/15 [09:23<00:00, 37.60s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist045_temp03.json

Średni czas odpowiedzi: 35.59s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, Yule revived th...",19.50,[He published in 3 papers between 1838 and 184...
1,What are the three components that make up the...,"Okay, I can help you understand the decomposit...",30.24,[lp us figure out how to minimize it.\nThis is...
2,"In the context of the Apriori algorithm, what ...","The 'downward closure' property of support, in...",56.44,[We want to find all strong rules. These are r...
3,Which R package is required to implement Suppo...,Based on the material provided:\n\nTo implemen...,19.13,"[nclude “base,” “utils,” “graphics,” and “stat..."
4,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,48.80,[]


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the course materials, Yule revived th...",[He published in 3 papers between 1838 and 184...,The name 'logistic' was revived by Yule in a p...,19.50,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you understand the decomposit...",[lp us figure out how to minimize it.\nThis is...,According to the Bias-Variance decomposition t...,30.24,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The 'downward closure' property of support, in...",[We want to find all strong rules. These are r...,Downward closure is a monotonicity property st...,56.44,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,Based on the material provided:\n\nTo implemen...,"[nclude “base,” “utils,” “graphics,” and “stat...",The 'e1071' package is the additional R packag...,19.13,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,[],"For classification splitting, the CART algorit...",48.80,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can help you understand the iterative ...",[The K-Means Algorithm\n\nChoose the value of ...,"Before starting, the user must choose the valu...",20.34,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,"That's a great question! You're right, high-di...",[e random process) of how x is generated.\n\nN...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,47.08,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the logic behind using the...",[Actually Patrons has the highest gain among t...,Information Gain tends to favor attributes tha...,36.96,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, I can help you understand why the mean i...","[While we’re analyzing, we’ll need to consider...",Lemma 1 proves that for any set C and any repr...,44.14,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with that!\n\nBased on th...","[Gin\n\nf p bet\n\nclassifica\n\ny,\n\nrop\n\n...","At each node, C4.5 recursively evaluates three...",28.35,None


In [6]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist07_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist07_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist07_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:08<00:00, 36.58s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist07_temp03.json

Średni czas odpowiedzi: 34.57s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",22.69,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,"Okay, I can help you understand the three comp...",24.58,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",44.48,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",18.88,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",43.60,[istance) to the centroid (want it to be small...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the provided text, Yule revived the n...",[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,22.69,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you understand the three comp...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,24.58,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The ""downward closure"" property of support, in...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,44.48,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,18.88,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[istance) to the centroid (want it to be small...,"For classification splitting, the CART algorit...",43.60,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...","[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",40.88,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,That's a great question! It touches on a pract...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,36.90,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The course materials explain that while Inform...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,25.03,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down why Lemma 1 tells us th...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,47.81,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with the options C4.5 con...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",22.41,None


In [7]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_llama_3_1_8B_instruct_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_llama_3_1_8B_instruct_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_llama_3_1_8B_instruct_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [10:02<00:00, 40.17s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_llama_3_1_8B_instruct_topk10_dist08_temp03.json

Średni czas odpowiedzi: 38.16s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"According to the course materials, the term ""l...",35.87,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,"Based on the provided course materials, the th...",25.02,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","In the context of the Apriori algorithm, the d...",46.82,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"According to the course materials, the require...",20.85,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"Based on the course materials, the specific no...",48.67,[istance) to the centroid (want it to be small...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"According to the course materials, the term ""l...",[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,35.87,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Based on the provided course materials, the th...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,25.02,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","In the context of the Apriori algorithm, the d...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,46.82,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"According to the course materials, the require...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,20.85,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"Based on the course materials, the specific no...",[istance) to the centroid (want it to be small...,"For classification splitting, the CART algorit...",48.67,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,I'd be happy to explain the two iterative step...,"[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",39.09,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,"According to the course materials, when dealin...",[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,43.84,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The Gain Ratio is an alternative to Informatio...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,23.88,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","According to Lemma 1, the cost function for a ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,50.85,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"According to the course materials, when decidi...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",24.49,None


In [8]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_mistral_nemo_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_mistral_nemo_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_mistral_nemo_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [10:07<00:00, 40.52s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_mistral_nemo_topk10_dist08_temp03.json

Średni czas odpowiedzi: 38.51s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,The term 'logistic' was revived in 1925 by Udn...,26.84,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,"Based on the provided context, the decompositi...",35.27,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...",The 'downward closure' property of support in ...,57.92,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Based on the provided course materials, here a...",30.35,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,The specific node impurity measure used by the...,40.43,[istance) to the centroid (want it to be small...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,The term 'logistic' was revived in 1925 by Udn...,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,26.84,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Based on the provided context, the decompositi...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,35.27,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...",The 'downward closure' property of support in ...,[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,57.92,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Based on the provided course materials, here a...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,30.35,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,The specific node impurity measure used by the...,[istance) to the centroid (want it to be small...,"For classification splitting, the CART algorit...",40.43,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,The K-Means algorithm is an iterative clusteri...,"[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",41.91,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,The Naïve Bayes algorithm handles this situati...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,29.82,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The logic behind using the 'Gain Ratio' instea...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,36.70,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","According to Lemma 1 in the clustering notes, ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,48.71,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,The three options that C4.5 considers when dec...,"[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",21.92,None


In [9]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gpt4o_mini_topk10_dist08_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gpt4o_mini_topk10_dist08_temp03'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gpt4o_mini_topk10_dist08_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [10:34<00:00, 42.31s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gpt4o_mini_topk10_dist08_temp03.json

Średni czas odpowiedzi: 40.30s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,The term 'logistic' was revived by Yule in a p...,24.95,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,The decomposition of the expected prediction e...,31.04,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...",The 'downward closure' property of support is ...,49.02,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"To implement Support Vector Machines (SVM), yo...",31.10,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,45.90,[istance) to the centroid (want it to be small...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,The term 'logistic' was revived by Yule in a p...,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,24.95,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,The decomposition of the expected prediction e...,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,31.04,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...",The 'downward closure' property of support is ...,[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,49.02,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"To implement Support Vector Machines (SVM), yo...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,31.10,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,[istance) to the centroid (want it to be small...,"For classification splitting, the CART algorit...",45.90,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,The K-Means algorithm consists of two main ite...,"[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",33.15,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,The Naïve Bayes algorithm addresses the challe...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,46.49,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The logic behind using the Gain Ratio instead ...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,34.51,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","According to Lemma 1, the mean is the optimal ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,38.48,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,C4.5 considers three options when deciding whe...,"[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",39.64,None


In [10]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp01" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp01'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp01.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): []...


Test progress: 100%|██████████| 15/15 [17:04<00:00, 68.28s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp01.json

Średni czas odpowiedzi: 66.27s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the materials I have, it was Raymond ...",60.83,[]
1,What are the three components that make up the...,"Okay, I can help you with that! Based on the c...",75.60,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The downward closure property of support, in t...",65.00,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",64.31,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",80.20,[istance) to the centroid (want it to be small...


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the materials I have, it was Raymond ...",[],The name 'logistic' was revived by Yule in a p...,60.83,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you with that! Based on the c...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,75.60,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The downward closure property of support, in t...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,65.00,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,64.31,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,"For classification tasks, the CART (Classifica...",[istance) to the centroid (want it to be small...,"For classification splitting, the CART algorit...",80.20,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can help you understand the K-Means al...","[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",58.30,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,That's a great question! It touches on a pract...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,64.26,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The course materials explain that while Inform...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,51.02,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down how Lemma 1 shows that ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,82.46,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with that! According to t...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",81.29,None


In [11]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp05" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp05'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp05.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [16:03<00:00, 64.21s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp05.json

Średni czas odpowiedzi: 62.20s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"According to the course materials, the name ""l...",70.63,[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...
1,What are the three components that make up the...,"Okay, I can help you with that! Based on the m...",66.52,"[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n..."
2,"In the context of the Apriori algorithm, what ...","The downward closure property of support, in t...",68.10,[Rule Mining and the Apriori Algorithm\n\nMIT ...
3,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",54.97,[R for Machine Learning\n\nAllison Chang\n\n1 ...
4,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,47.87,[]


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"According to the course materials, the name ""l...",[f(xi)\n• “logistic loss” log2 1 + e\n⇐= logis...,The name 'logistic' was revived by Yule in a p...,70.63,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you with that! Based on the m...","[• Input: {(xi, yi)}m\n\ni=1, xi ∈X , yi ∈ R\n...",According to the Bias-Variance decomposition t...,66.52,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The downward closure property of support, in t...",[Rule Mining and the Apriori Algorithm\n\nMIT ...,Downward closure is a monotonicity property st...,68.10,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Okay, I can help you with that!\n\nBased on th...",[R for Machine Learning\n\nAllison Chang\n\n1 ...,The 'e1071' package is the additional R packag...,54.97,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,[],"For classification splitting, the CART algorit...",47.87,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can definitely help you understand the...","[ame(cbind(y,x1,x2)))\n\n4.4\nK-Means Clusteri...","Before starting, the user must choose the valu...",52.80,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,That's a great question! It touches on a pract...,[4.6 Na¨ıve Bayes\n\nInstall and load the e107...,Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,62.82,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,The course materials explain that while Inform...,"[-section risk, etc.\n\nHow to build a decisio...",Information Gain tends to favor attributes tha...,78.47,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down how Lemma 1 shows that ...",[an objects assigned to different clusters. We...,Lemma 1 proves that for any set C and any repr...,53.80,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can explain the three options C4.5 con...","[-section risk, etc.\n\nHow to build a decisio...","At each node, C4.5 recursively evaluates three...",52.98,None


In [12]:
RUN_LABEL = "statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp07" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics")

Test Start: 'statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp07'
Results will be saved in: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp07.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): []...


Test progress: 100%|██████████| 15/15 [12:37<00:00, 50.49s/it]

Success! JSON saved at: statistics\results_statistics_results_chunk_512_overlap_100_text_to_embedding_small_gemini20_flash_topk10_dist08_temp07.json

Średni czas odpowiedzi: 48.47s


,question,answer,latency,contexts
0,Who revived the term 'logistic' in 1925 to des...,"Based on the materials I have, it was Raymond ...",92.86,[]
1,What are the three components that make up the...,"Okay, I can help you with that!\n\nThe decompo...",52.50,[]
2,"In the context of the Apriori algorithm, what ...","The 'downward closure' property of support, in...",54.19,[]
3,Which R package is required to implement Suppo...,"Based on the tutorial content, the `e1071` pac...",43.79,[]
4,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,42.09,[]


,config_label,question,answer,contexts,ground_truth,latency,error
0,statistics_results_chunk_512_overlap_100_text_...,Who revived the term 'logistic' in 1925 to des...,"Based on the materials I have, it was Raymond ...",[],The name 'logistic' was revived by Yule in a p...,92.86,None
1,statistics_results_chunk_512_overlap_100_text_...,What are the three components that make up the...,"Okay, I can help you with that!\n\nThe decompo...",[],According to the Bias-Variance decomposition t...,52.50,None
2,statistics_results_chunk_512_overlap_100_text_...,"In the context of the Apriori algorithm, what ...","The 'downward closure' property of support, in...",[],Downward closure is a monotonicity property st...,54.19,None
3,statistics_results_chunk_512_overlap_100_text_...,Which R package is required to implement Suppo...,"Based on the tutorial content, the `e1071` pac...",[],The 'e1071' package is the additional R packag...,43.79,None
4,statistics_results_chunk_512_overlap_100_text_...,What specific node impurity measure is used by...,The CART (Classification and Regression Trees)...,[],"For classification splitting, the CART algorit...",42.09,None
5,statistics_results_chunk_512_overlap_100_text_...,Explain the two iterative steps of the K-Means...,"Okay, I can certainly help you understand the ...",[],"Before starting, the user must choose the valu...",22.65,None
6,statistics_results_chunk_512_overlap_100_text_...,How does the Naïve Bayes algorithm handle high...,That's a great question! It touches on a commo...,[],Naïve Bayes uses 'Laplace smoothing' (a Bayesi...,62.80,None
7,statistics_results_chunk_512_overlap_100_text_...,Describe the logic behind using the 'Gain Rati...,"Okay, I can explain the logic behind using Gai...",[],Information Gain tends to favor attributes tha...,54.52,None
8,statistics_results_chunk_512_overlap_100_text_...,"According to Lemma 1 in the clustering notes, ...","Okay, let's break down why Lemma 1 in the clus...",[],Lemma 1 proves that for any set C and any repr...,22.25,None
9,statistics_results_chunk_512_overlap_100_text_...,What are the three options C4.5 considers when...,"Okay, I can help you with that!\n\nBased on th...",[],"At each node, C4.5 recursively evaluates three...",46.88,None


## Chat prompts tests
fixed parameters: CHAT_MODEL="google/gemini-2.0-flash-001", EMBED_MODEL="text-embedding-3-small", MAX_TOKENS="1024", RAG_CHUNK_OVERLAP="100", RAG_CHUNK_SIZE="512", RAG_DISTANCE_THRESHOLD="0.8"
,RAG_TOP_K="10", TEMPERATURE="0.3"


In [5]:
RUN_LABEL = "chat_prompt_baseline" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

Test Start: 'chat_prompt_baseline'
Results will be saved in: statistics/prompts\results_chat_prompt_baseline.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:49<00:00, 39.31s/it]


Success! JSON saved at: statistics/prompts\results_chat_prompt_baseline.json


TypeError: list indices must be integers or slices, not str

In [7]:
RUN_LABEL = "chat_prompt_fewshot" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\4181456619.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_fewshot'
Results will be saved in: statistics\prompts\results_chat_prompt_fewshot.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:37<00:00, 38.48s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_fewshot.json


In [8]:
RUN_LABEL = "chat_prompt_cot" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\4120203572.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_cot'
Results will be saved in: statistics\prompts\results_chat_prompt_cot.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:37<00:00, 38.48s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_cot.json


In [9]:
RUN_LABEL = "chat_prompt_socratic" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\1766480541.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_socratic'
Results will be saved in: statistics\prompts\results_chat_prompt_socratic.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:22<00:00, 37.52s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_socratic.json


In [10]:
RUN_LABEL = "chat_prompt_retrieval-optimized" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\1676728088.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_retrieval-optimized'
Results will be saved in: statistics\prompts\results_chat_prompt_retrieval-optimized.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:37<00:00, 38.51s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_retrieval-optimized.json


In [11]:
RUN_LABEL = "chat_prompt_error_correcting" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\2218749680.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_error_correcting'
Results will be saved in: statistics\prompts\results_chat_prompt_error_correcting.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:37<00:00, 38.47s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_error_correcting.json


In [12]:
RUN_LABEL = "chat_prompt_adaptive" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\2537570332.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_adaptive'
Results will be saved in: statistics\prompts\results_chat_prompt_adaptive.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:34<00:00, 38.31s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_adaptive.json


In [13]:
RUN_LABEL = "chat_prompt_analogy" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_25472\262586300.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts"


Test Start: 'chat_prompt_analogy'
Results will be saved in: statistics\prompts\results_chat_prompt_analogy.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['f(xi)\n• “logistic loss” log2 1 + e\n⇐= logistic regression\n\n6\n\n[IMAGE_OCR 1]\n35 — tj <0)\nexp)\n3 04,1 +exp yo)\n—— max(0,1-¥"09)\n25 2 15 1\nu = SS\n==)\no\n-2 AS A -0.5 o 0.5 1 15 2\ny* F090...


Test progress: 100%|██████████| 15/15 [09:50<00:00, 39.34s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_analogy.json


In [5]:
RUN_LABEL = "chat_prompt_mode_quick" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts",
    mode="quick"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_9000\1008356450.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts",


Test Start: 'chat_prompt_mode_quick'
Results will be saved in: statistics\prompts\results_chat_prompt_mode_quick.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['Logistic Regression\n\nMIT 15.097 Course Notes Cynthia Rudin\n\nThanks to Ashia Wilson Credit: J.S. Cramer’s “The Origin of Logistic Regression”\n\nOrigins: 19th Century.\n\n• Studying growth of pop...


Test progress:  40%|████      | 6/15 [05:26<08:26, 56.32s/it]


SERVER ERROR: 502 for question: How does the Naïve Bayes algorithm handle high-dimensional data where a specific feature value might not appear in the training set for a given class?


Test progress: 100%|██████████| 15/15 [12:35<00:00, 50.38s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_mode_quick.json


In [6]:
RUN_LABEL = "chat_prompt_mode_deep" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts",
    mode="deep"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_9000\172665986.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts",


Test Start: 'chat_prompt_mode_deep'
Results will be saved in: statistics\prompts\results_chat_prompt_mode_deep.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['Logistic Regression\n\nMIT 15.097 Course Notes Cynthia Rudin\n\nThanks to Ashia Wilson Credit: J.S. Cramer’s “The Origin of Logistic Regression”\n\nOrigins: 19th Century.\n\n• Studying growth of pop...


Test progress: 100%|██████████| 15/15 [14:47<00:00, 59.16s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_mode_deep.json


In [7]:
RUN_LABEL = "chat_prompt_mode_coach" 

df_results = run_notebook_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="statistics\prompts",
    mode="coach"
)

<>:9: SyntaxWarning: invalid escape sequence '\p'
<>:9: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anost\AppData\Local\Temp\ipykernel_9000\474207931.py:9: SyntaxWarning: invalid escape sequence '\p'
  output_folder="statistics\prompts",


Test Start: 'chat_prompt_mode_coach'
Results will be saved in: statistics\prompts\results_chat_prompt_mode_coach.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['Logistic Regression\n\nMIT 15.097 Course Notes Cynthia Rudin\n\nThanks to Ashia Wilson Credit: J.S. Cramer’s “The Origin of Logistic Regression”\n\nOrigins: 19th Century.\n\n• Studying growth of pop...


Test progress: 100%|██████████| 15/15 [14:28<00:00, 57.93s/it]

Success! JSON saved at: statistics\prompts\results_chat_prompt_mode_coach.json


## quiz prompts

In [10]:
QUIZ_API_URL = "https://chatbot-backend-production-4b66.up.railway.app/quiz/generate/11"
QUIZ_COURSE_ID = "11"
QUIZ_NUM_QUESTIONS = 10

In [11]:
import datetime as dt

In [12]:
def run_quiz_test_final(run_label, api_url, num_questions=10, topic=None, moodle_token=None, material_ids=None):
    """
    Final quiz test function - tests general (full course), topic-specific, lub quiz z wybranych materiałów.
    Args:
        topic: quiz tematyczny (None = quiz ogólny)
        material_ids: lista ID materiałów (np. [5,6]) - jeśli podana, quiz tylko z tych materiałów
    """
    import os

    file_tag = f"_{run_label}" if run_label else ""
    output_folder = "prompts_quizzes"
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f"quiz_results{file_tag}.json")
    print(f"Quiz Test Start '{run_label}'")
    print(f"Results will be saved in file: {output_filename}")
    results = []
    headers = {"Content-Type": "application/json"}
    if moodle_token:
        headers["Authorization"] = f"Bearer {moodle_token}"

    # Quiz z wybranych materiałów (np. 5 i 6)
    if material_ids is not None:
        print(f"Generating quiz ONLY for materials: {material_ids} ...")
        payload = {
            "materialIds": material_ids,
            "numQuestions": num_questions
        }
        start_time = time.time()
        quiz_data = None
        error_msg = None
        duration = 0
        try:
            response = requests.post(api_url, json=payload, headers=headers, timeout=300)
            if response.status_code == 200:
                duration = time.time() - start_time
                quiz_data = response.json()
            else:
                error_msg = f"HTTP {response.status_code}"
                print(f"SERVER ERROR: {response.status_code} for materials {material_ids} quiz")
        except Exception as e:
            error_msg = str(e)
            duration = 300
            print(f"\nTimeout for materials {material_ids} quiz")
        results.append({
            "config_label": run_label,
            "test_type": f"materials_{'_'.join(map(str, material_ids))}",
            "materialIds": material_ids,
            "quiz_data": quiz_data,
            "num_questions_requested": num_questions,
            "num_questions_generated": len(quiz_data.get("questions", [])) if quiz_data else 0,
            "latency": round(duration, 2),
            "error": error_msg
        })
        print("Generating quiz 2/3: Full Course (General)...")
        payload_general = {
            "materialIds": [],
            "topic": topic,
            "numQuestions": num_questions
        }
        start_time = time.time()
        quiz_data = None
        error_msg = None
        duration = 0
        try:
            response = requests.post(api_url, json=payload_general, headers=headers, timeout=300)
            if response.status_code == 200:
                duration = time.time() - start_time
                quiz_data = response.json()
            else:
                error_msg = f"HTTP {response.status_code}"
                print(f"SERVER ERROR: {response.status_code} for general quiz")
        except Exception as e:
            error_msg = str(e)
            duration = 300
            print(f"\nTimeout for general quiz")
        results.append({
            "config_label": run_label,
            "test_type": "full_course",
            "topic": None,
            "quiz_data": quiz_data,
            "num_questions_requested": num_questions,
            "num_questions_generated": len(quiz_data.get("questions", [])) if quiz_data else 0,
            "latency": round(duration, 2),
            "error": error_msg
        })
        time.sleep(3.0)
        # Test 2: Topic-specific quiz
        if topic:
            print(f"Generating quiz 3/3: Topic '{topic}' ...")
            payload_topic = {
                "materialIds": [],
                "topic": topic,
                "numQuestions": num_questions
            }
            start_time = time.time()
            quiz_data = None
            error_msg = None
            duration = 0
            try:
                response = requests.post(api_url, json=payload_topic, headers=headers, timeout=300)
                if response.status_code == 200:
                    duration = time.time() - start_time
                    quiz_data = response.json()
                else:
                    error_msg = f"HTTP {response.status_code}"
                    print(f"SERVER ERROR: {response.status_code} for topic: {topic}")
            except Exception as e:
                error_msg = str(e)
                duration = 300
                print(f"\nTimeout for topic: {topic}")
            results.append({
                "config_label": run_label,
                "test_type": "topic",
                "topic": topic,
                "quiz_data": quiz_data,
                "num_questions_requested": num_questions,
                "num_questions_generated": len(quiz_data.get("questions", [])) if quiz_data else 0,
                "latency": round(duration, 2),
                "error": error_msg
            })
            time.sleep(3.0)

    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    df = pd.DataFrame(results)
    try:
        import openpyxl
        excel_name = output_filename.replace(".json", ".xlsx")
        df.to_excel(excel_name, index=False)
        print(f"Success! Results saved in file: {excel_name}")
    except Exception as e:
        print(f"Error saving Excel file: {e}")
    return df


In [29]:
FINAL_TOPIC = "decision trees"
FINAL_NUM_QUESTIONS = 15

In [30]:
RUN_LABEL = "quiz_prompt_baseline" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url=QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_baseline'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_baseline.json
Generating quiz ONLY for materials: [5, 6] ...
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_baseline.xlsx


In [31]:
RUN_LABEL = "quiz_prompt_constrained" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url = QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_constrained'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_constrained.json
Generating quiz ONLY for materials: [5, 6] ...
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_constrained.xlsx


In [32]:
RUN_LABEL = "quiz_prompt_socratic" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url = QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_socratic'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_socratic.json
Generating quiz ONLY for materials: [5, 6] ...
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_socratic.xlsx


In [33]:
RUN_LABEL = "quiz_prompt_verbose" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url = QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_verbose'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_verbose.json
Generating quiz ONLY for materials: [5, 6] ...
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_verbose.xlsx


In [34]:
RUN_LABEL = "quiz_prompt_blooms_taxonomy" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url = QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_blooms_taxonomy'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_blooms_taxonomy.json
Generating quiz ONLY for materials: [5, 6] ...
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_blooms_taxonomy.xlsx


In [35]:
RUN_LABEL = "quiz_prompt_quiz_scenario" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url = QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_quiz_scenario'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_quiz_scenario.json
Generating quiz ONLY for materials: [5, 6] ...
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_quiz_scenario.xlsx


In [37]:
RUN_LABEL = "quiz_prompt_quiz_misconception" 
df_results = run_quiz_test_final(
    run_label = RUN_LABEL,
    api_url = QUIZ_API_URL, 
    num_questions=FINAL_NUM_QUESTIONS, 
    topic=FINAL_TOPIC, 
    moodle_token=MOODLE_TOKEN, 
    material_ids=[5,6])

Quiz Test Start 'quiz_prompt_quiz_misconception'
Results will be saved in file: prompts_quizzes\quiz_results_quiz_prompt_quiz_misconception.json
Generating quiz ONLY for materials: [5, 6] ...
SERVER ERROR: 404 for materials [5, 6] quiz
Generating quiz 2/3: Full Course (General)...
Generating quiz 3/3: Topic 'decision trees' ...
SERVER ERROR: 404 for topic: decision trees
Success! Results saved in file: prompts_quizzes\quiz_results_quiz_prompt_quiz_misconception.xlsx


# philosophy course

In [12]:
API_URL = "https://chatbot-backend-production-4b66.up.railway.app/chat"  # Lub adres Railway
TEST_COURSE_ID = "15"                      # ID kursu z materiałami
MOODLE_TOKEN = '9f66de570d71c9b474cce459dd02a2bd'               # Opcjonalnie token
INPUT_FILE = "question_answer_philosophy.json"

In [13]:
RUN_LABEL = "philosophy_results_chunk_512_overlap_100_voyage3lite_gemini20_flash_topk10_dist05_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="philosophy")

Test Start: 'philosophy_results_chunk_512_overlap_100_voyage3lite_gemini20_flash_topk10_dist05_temp03'
Results will be saved in: philosophy\results_philosophy_results_chunk_512_overlap_100_voyage3lite_gemini20_flash_topk10_dist05_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ["ivine knowledge. Socrates' human wisdom is to acknowledge this fallibility in the elenchus, which is the best procedure we humans have to achieve knowledge.\n\n24.01\n\n3", 'sicians, others as carpe...


Test progress: 100%|██████████| 15/15 [16:11<00:00, 64.74s/it]

Success! JSON saved at: philosophy\results_philosophy_results_chunk_512_overlap_100_voyage3lite_gemini20_flash_topk10_dist05_temp03.json

Średni czas odpowiedzi: 62.74s


,question,answer,latency,contexts
0,What is the Greek word for 'happiness' accordi...,"According to the provided text, the Greek word...",58.94,[ivine knowledge. Socrates' human wisdom is to...
1,What are the birth and death years of René Des...,Based on the provided text:\n\n* René Descar...,73.59,[Cartesian Doubt in Meditation 1\n\nI. Backgro...
2,"According to Book I, Chapter 7 of the Nichomac...","Okay, I can help you with that!\n\nAccording t...",58.81,[ just/unjust.\nApology: 28b\nCrito: 48c-d\n\n...
3,Define the 'articulation condition' as it rela...,"Okay, I can certainly help with that!\n\nBased...",69.48,[areness of how little he does know. If we all...
4,What is Aristotle's formal definition of the s...,Aristotle defines the soul (psuchē) in 'De Ani...,49.29,[rtance and the route to happiness. In contras...


,config_label,question,answer,contexts,ground_truth,latency,error
0,philosophy_results_chunk_512_overlap_100_voyag...,What is the Greek word for 'happiness' accordi...,"According to the provided text, the Greek word...",[ivine knowledge. Socrates' human wisdom is to...,The Greek word for 'happiness' is 'eudaimonia'...,58.94,None
1,philosophy_results_chunk_512_overlap_100_voyag...,What are the birth and death years of René Des...,Based on the provided text:\n\n* René Descar...,[Cartesian Doubt in Meditation 1\n\nI. Backgro...,René Descartes was born in 1596 and died in 16...,73.59,None
2,philosophy_results_chunk_512_overlap_100_voyag...,"According to Book I, Chapter 7 of the Nichomac...","Okay, I can help you with that!\n\nAccording t...",[ just/unjust.\nApology: 28b\nCrito: 48c-d\n\n...,The three stages are: (i) the existence of a h...,58.81,None
3,philosophy_results_chunk_512_overlap_100_voyag...,Define the 'articulation condition' as it rela...,"Okay, I can certainly help with that!\n\nBased...",[areness of how little he does know. If we all...,The articulation condition states that if a pe...,69.48,None
4,philosophy_results_chunk_512_overlap_100_voyag...,What is Aristotle's formal definition of the s...,Aristotle defines the soul (psuchē) in 'De Ani...,[rtance and the route to happiness. In contras...,Aristotle defines the soul as the 'first actua...,49.29,None
5,philosophy_results_chunk_512_overlap_100_voyag...,Explain Aristotle's reasoning for why a well-o...,"Okay, I can explain Aristotle's reasoning for ...",[rtance and the route to happiness. In contras...,Aristotle argues that if we do not choose thin...,71.17,None
6,philosophy_results_chunk_512_overlap_100_voyag...,Describe Descartes' 'Dream Argument' and expla...,"Okay, I can certainly help you understand Desc...",[],"Descartes notes that during sleep, he is often...",58.34,None
7,philosophy_results_chunk_512_overlap_100_voyag...,"What is the 'Socratic Elenchus', and what are ...","Based on the provided text, the Socratic Elenc...",[24.01: Classics of Western Philosophy Prof. S...,The Socratic elenchus is a standard pattern of...,79.29,None
8,philosophy_results_chunk_512_overlap_100_voyag...,How does Aristotle use the comparison of an ax...,Aristotle uses the comparison of an axe to ill...,[rtance and the route to happiness. In contras...,Aristotle suggests that if an axe were a natur...,55.56,None
9,philosophy_results_chunk_512_overlap_100_voyag...,Why does Socrates argue in the 'Meno' that no ...,"Okay, that's a great question! Let's explore w...",[],Socrates argues that people only desire things...,50.39,None


In [6]:
RUN_LABEL = "philosophy_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist05_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="philosophy")


Test Start: 'philosophy_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist05_temp03'
Results will be saved in: philosophy\results_philosophy_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist05_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): ['ny of us are irrational and our lives are a mess because our lives are not ordered around a supreme good. The goal of his project is to figure out if there is a single supreme good that is good for ...


Test progress: 100%|██████████| 15/15 [15:13<00:00, 60.87s/it]


Success! JSON saved at: philosophy\results_philosophy_results_chunk_1000_overlap_200_voyage3lite_gemini20_flash_topk10_dist05_temp03.json


UnboundLocalError: local variable 'df_results' referenced before assignment

In [14]:
RUN_LABEL = "philosophy_results_chunk_2000_overlap_400_voyage3lite_gemini20_flash_topk10_dist05_temp03" 
run_pipeline_test(
    run_label=RUN_LABEL,
    api_url=API_URL,
    course_id=TEST_COURSE_ID,
    input_file=INPUT_FILE,
    moodle_token=MOODLE_TOKEN,
    output_folder="philosophy")

Test Start: 'philosophy_results_chunk_2000_overlap_400_voyage3lite_gemini20_flash_topk10_dist05_temp03'
Results will be saved in: philosophy\results_philosophy_results_chunk_2000_overlap_400_voyage3lite_gemini20_flash_topk10_dist05_temp03.json


Test progress:   0%|          | 0/15 [00:00<?, ?it/s]


 First context preview (truncated): [" desirable and lacking in\nnothing; and such we think happiness to be.” (1097b:18)\n\nBut the story doesn't stop when we conclude that happiness is the supreme good. Everyone may agree on this,\n\nP...


Test progress: 100%|██████████| 15/15 [16:16<00:00, 65.12s/it]

Success! JSON saved at: philosophy\results_philosophy_results_chunk_2000_overlap_400_voyage3lite_gemini20_flash_topk10_dist05_temp03.json

Średni czas odpowiedzi: 63.12s


,question,answer,latency,contexts
0,What is the Greek word for 'happiness' accordi...,"I am sorry, but the text you have provided doe...",50.16,[ desirable and lacking in\nnothing; and such ...
1,What are the birth and death years of René Des...,Based on the course materials:\n\nRené Descart...,76.00,[Cartesian Doubt in Meditation 1\n\nI. Backgro...
2,"According to Book I, Chapter 7 of the Nichomac...","Okay, I can help you with that!\n\nAccording t...",91.80,"[Aristotle on Happiness Nichomachean Ethics, B..."
3,Define the 'articulation condition' as it rela...,"Okay, I can help with that!\n\nBased on the co...",56.14,[24.01: Classics of Western Philosophy Prof. S...
4,What is Aristotle's formal definition of the s...,"Based on the provided text, Aristotle defines ...",60.21,[24.01: Classics of Western Philosophy Prof. S...


,config_label,question,answer,contexts,ground_truth,latency,error
0,philosophy_results_chunk_2000_overlap_400_voya...,What is the Greek word for 'happiness' accordi...,"I am sorry, but the text you have provided doe...",[ desirable and lacking in\nnothing; and such ...,The Greek word for 'happiness' is 'eudaimonia'...,50.16,None
1,philosophy_results_chunk_2000_overlap_400_voya...,What are the birth and death years of René Des...,Based on the course materials:\n\nRené Descart...,[Cartesian Doubt in Meditation 1\n\nI. Backgro...,René Descartes was born in 1596 and died in 16...,76.00,None
2,philosophy_results_chunk_2000_overlap_400_voya...,"According to Book I, Chapter 7 of the Nichomac...","Okay, I can help you with that!\n\nAccording t...","[Aristotle on Happiness Nichomachean Ethics, B...",The three stages are: (i) the existence of a h...,91.80,None
3,philosophy_results_chunk_2000_overlap_400_voya...,Define the 'articulation condition' as it rela...,"Okay, I can help with that!\n\nBased on the co...",[24.01: Classics of Western Philosophy Prof. S...,The articulation condition states that if a pe...,56.14,None
4,philosophy_results_chunk_2000_overlap_400_voya...,What is Aristotle's formal definition of the s...,"Based on the provided text, Aristotle defines ...",[24.01: Classics of Western Philosophy Prof. S...,Aristotle defines the soul as the 'first actua...,60.21,None
5,philosophy_results_chunk_2000_overlap_400_voya...,Explain Aristotle's reasoning for why a well-o...,"Okay, let's break down Aristotle's argument fo...","[Matter and Form, Body and Soul, in De Anima\n...",Aristotle argues that if we do not choose thin...,64.57,None
6,philosophy_results_chunk_2000_overlap_400_voya...,Describe Descartes' 'Dream Argument' and expla...,"Okay, let's break down Descartes' ""Dream Argum...","[set of beliefs by rejecting one which is, as ...","Descartes notes that during sleep, he is often...",54.50,None
7,philosophy_results_chunk_2000_overlap_400_voya...,"What is the 'Socratic Elenchus', and what are ...","Based on the provided text, the Socratic elenc...",[24.01: Classics of Western Philosophy Prof. S...,The Socratic elenchus is a standard pattern of...,76.94,None
8,philosophy_results_chunk_2000_overlap_400_voya...,How does Aristotle use the comparison of an ax...,Aristotle uses the analogy of an axe to explai...,[24.01: Classics of Western Philosophy Prof. S...,Aristotle suggests that if an axe were a natur...,54.34,None
9,philosophy_results_chunk_2000_overlap_400_voya...,Why does Socrates argue in the 'Meno' that no ...,Socrates' argument in the *Meno* that no one d...,[d) Socrates knows that he does not have knowl...,Socrates argues that people only desire things...,52.26,None


# statistics course